In [ ]:

type SGLD_state
    w::Array{Float64,1} # vector state
    eps::Float64
    function SGLD_specs(w,eps)
      new(w,eps)
    end
end
# stochastic gradient langevin dynamics
# grad is a function which returns stochastic gradient given x as input
function sgld!(s::SGLD_state, grad::Function)
  # stochastic gradient langevin dynamics
  s.w[:] += s.eps.* grad(s.w)/2 + sqrt(2.0*s.eps).*randn(length(s.w))
end

type SGLDERM_state
    U::Array{Float64,2} # matrix state
    eps::Float64
    function SGLDERM_specs(U,eps)
        new(U,eps)
    end
end

function sglderm!(s::SGLDERM_state,grad::Function,proj::Function,geod::Function)
    # stochastic gradient langevin dynamics for embedded riemannian manifolds
    # proj(U,.) is the orthogonal projection onto the tangent space at U.
    # geod(U,mom,eps) is the end point of the geodesic flow 
    # initialised by (U,mom) taken for time eps.
    mom=proj(s.U,sqrt(s.eps).*grad(s.U)/2+randn(size(s.U)))
    s.U[:]+=geod(s.U,mom,sqrt(s.eps))
end  